In [1]:
import pickle
import matplotlib.pyplot as plt
import numpy as np
import random
import time
import tensorflow as tf
from AADatasets import *
from AAPreTrain import *
from AATransferLearn import *
from AAlogic import *
from LabnotesDoc import *

In [2]:
def run_target(params):
    x,y = get_data(params)
    # x,y = keep_class(x,y,[0,1,2])
    # x,y = equal_data_min(x,y)

    x_test,y_test,x,y = val_split(x,y, params["test_size"])
    x_val,y_val,x,y = val_split(x,y, params["val_size"])
    
    # for method in m:
    config_desktop()
#     gpu_options = tf.GPUOptions(per_process_gpu_memory_fraction=0.45)

    model = make_model(x, y, params)

    if params["style"] == 'FT':
        weights = determen_weights(y)
        H, score, model = train_model(model,x,y,x_val,y_val,x_test,y_test, params["epochs"], params["Batch_size"])
        predictions = get_feature_vector(model, x, layer = 'fc2')
        predictions_test = get_feature_vector(model, x_test, layer = 'fc2')
        score = auc_svm(predictions,y,predictions_test,y_test, plot = False)
        results = {'score':score,"acc_epoch":H.history['acc'],"val_acc_epoch":H.history['val_acc'],"loss_epoch":H.history['loss'],"vall_loss_epoch":H.history['val_loss']}

    elif params["style"] =='SVM':
        predictions = get_feature_vector(model, x, layer = 'fc2')
        predictions_test = get_feature_vector(model, x_test, layer = 'fc2')
        score = auc_svm(predictions,y,predictions_test,y_test, plot = False)
        results = {'score':score,'data_name':params["data_name"],'method':params["model"],'style':params["style"]}
        H=None

    doc(params,results,H,params["doc_path"])

In [13]:
params = {"Data":'Breast',
                "data_name":None,
                "style":'FT',
                "model":"imagenet",
                "file_path":r"C:\breast-ultrasound-image",
                "pickle_path":r"C:\pickles\Breast",
                "model_path":{"KaggleDR":r"C:\models\Epochs_50_kaggleDR.json","Chest":r"C:\models\Epochs_50_Chest.json", "CatDog":r"C:\models\Epochs_40_CatDog.json" },
                "RandomSeed":2,
                "doc_path":r"C:\Users\Flori\Documents\GitHub\t",
                'img_size_x':224,
                'img_size_y':224,
                'norm':False,
                'color':True, 
                'pretrain':None, 
                "equal_data":False, 
                "shuffle":True, 
                "epochs":1 , 
                "val_size":25,
                "test_size":50, 
                "Batch_size":4
                }
params = {"Data":'Blood',
        "data_name":None,
        "style":"FT",
        "model":"imagenet",
        "file_path":r"C:\blood-cells",
        "pickle_path":r"C:\pickles\Blood",
        "model_path":{"KaggleDR":r"C:\models\Epochs_50_kaggleDR.json","Chest":r"C:\models\Epochs_50_Chest.json", "CatDog":r"C:\models\Epochs_40_CatDog.json" },
        "RandomSeed":2,
        "doc_path":r"C:\Users\Flori\Documents\GitHub\t",
        'img_size_x':224,
        'img_size_y':224,
        'norm':False,
        'color':True, 
        'pretrain':None, 
        "equal_data":False, 
        "shuffle":True, 
        "epochs":50 , 
        "val_size":300,
        "test_size":500, 
        "Batch_size":32
        }

In [ ]:
from AATransferLearn import *
run_target(params)

Try to import pickle
succeed to import pickle
 unzip
MODEL SUMMARY:
<tensorflow.python.keras._impl.keras.engine.topology.InputLayer object at 0x00000269C327BD68> True
<tensorflow.python.keras._impl.keras.layers.convolutional.Conv2D object at 0x00000269A10B9320> True
<tensorflow.python.keras._impl.keras.layers.convolutional.Conv2D object at 0x000002699FACD860> True
<tensorflow.python.keras._impl.keras.layers.pooling.MaxPooling2D object at 0x00000269C4EE7208> True
<tensorflow.python.keras._impl.keras.layers.convolutional.Conv2D object at 0x0000026A0C63D080> True
<tensorflow.python.keras._impl.keras.layers.convolutional.Conv2D object at 0x0000026A0C637240> True
<tensorflow.python.keras._impl.keras.layers.pooling.MaxPooling2D object at 0x0000026A0C6425C0> True
<tensorflow.python.keras._impl.keras.layers.convolutional.Conv2D object at 0x0000026A0C653CF8> True
<tensorflow.python.keras._impl.keras.layers.convolutional.Conv2D object at 0x0000026A0C647278> True
<tensorflow.python.keras._impl.ke

In [15]:
import numpy as np
import matplotlib.pyplot as plt
import cv2
import sklearn
from sklearn import datasets, svm, metrics
import os
import random
import pandas
import time
import pickle

def loading(size,i,start, name):
    stop = time.time()
    part = int(((i+1)/size)*20)
    loading_bar = part*'-'+(20-part)*' '
    print(f"{name}: {i+1}/{size}: [{loading_bar[0:10]}{part*5}%{loading_bar[10:20]}] elapsed time: {int(stop-start)}",end='\r')

def import_dogcat(path, img_size_x,img_size_y, norm, color):
    #DIR = r"C:\Users\Floris\Documents\Python scripts\PetImages"
    try: 
        cat = list(os.listdir(path))
        print('Directory found')
    except:
        print('Directory not Found')


    training_data = list()
    training_class = list()
    
    for category in cat:
        class_num = cat.index(category)
        path2 = os.path.join(path, category)
        start = time.time()
        i = 0
        size = len(list(os.listdir(path2)))
        for img in os.listdir(path2):
            try:
                if color:
                    D = 3
                    img_array = cv2.imread(os.path.join(path2,img), cv2.IMREAD_COLOR)
                    new_array = cv2.resize(img_array,(img_size_x, img_size_y))
                    
                else:
                    D = 1
                    img_array = cv2.imread(os.path.join(path2,img), cv2.IMREAD_GRAYSCALE)
                    new_array = cv2.resize(img_array,(img_size_x, img_size_y))
                training_data.append(new_array)
                if class_num == 0:
                    training_class.append([1,0])
                elif class_num == 1:
                    training_class.append([0,1])
            except Exception as e:
                pass
            loading(size,i,start, "Cat_Dog data import")
            i+=1
        print("\n")

    zip_list = list(zip(training_data,training_class))
#     random.shuffle(zip_list)
    training_data,training_class = zip(*zip_list)
    x = np.array(training_data).reshape(-1,img_size_x, img_size_y,D)
    y = np.array(training_class).reshape(-1,len(cat))

    if type(norm) != bool:
        print("please enter 'boolean' for norm(alization)")

    if norm:
        x = x/ 255.0

    print(f"This Dog_Cat dataset contains the following: \nTotal length Dataset = {len(x)} ")
    return x, y


random.seed(2)

print("Try to import pickle")
if params["Data"] == 'ISIC':
    zippy = list(pickle.load(open( f"{params['pickle_path']}{params['data_name']}.p", "rb" )))
else:
    try:
        zippy = list(pickle.load(open( f"{params['pickle_path']}.p", "rb" )))
    except:
        zippy = list(pickle.load(open( f"{params['pickle_path']}_part1.p", "rb" )))
        zippy2 = list(pickle.load(open( f"{params['pickle_path']}_part2.p", "rb" )))
        zippy.extend(zippy2)
print("succeed to import pickle")

print(" unzip")
random.shuffle(zippy)
x,y = zip(*zippy)
x = np.array(x)
y = np.array(y)


# x,y = import_dogcat(params['file_path'], params['img_size_x'],params['img_size_y'], norm = params["norm"], color = params["color"])

Try to import pickle
succeed to import pickle
 unzip


In [16]:
def count_classes(Y):
    '''
    Counts classes, no matter wich type of class notation it is, array of 1 and 0 or interger.
    input: class list
    output: class dict, {class:count}
    '''
    ## checks if they are intergers ##
    
    back_to_num = list()
    list_classes = list(Y)
    for i in list_classes:
        back_to_num.append(list(i).index(1))
        
    d = dict()
    for n in back_to_num:
        if n in d:
            d[n] += 1
        else:
            d[n] = 1
    return d

count_classes(y)

{1: 3109, 3: 3171, 0: 3133, 2: 3102}

In [18]:
def val_split(x,y, val_size):
    return x[:val_size] ,y[:val_size] ,x[val_size:] ,y[val_size:]

x_test,y_test,x,y = val_split(x,y, params["test_size"])
x_val,y_val,x,y = val_split(x,y, params["val_size"])

In [21]:
count_classes(y_val)
count_classes(y)
x.max()

255

In [3]:
x,y = get_data(params)
x_test,y_test,x,y = val_split(x,y, params["test_size"])
x_val,y_val,x,y = val_split(x,y, params["val_size"])

# for method in m:
config_desktop()

model = make_model(x, y, params)

weights = determen_weights(y)
H, score, model = train_model(model,x,y,x_val,y_val,x_test,y_test, params["epochs"], params["Batch_size"])
predictions = get_feature_vector(model, x, layer = 'fc2')
predictions_test = get_feature_vector(model, x_test, layer = 'fc2')


Try to import pickle
succeed to import pickle
 unzip
MODEL SUMMARY:
<tensorflow.python.keras._impl.keras.engine.topology.InputLayer object at 0x00000115C19DE3C8> True
<tensorflow.python.keras._impl.keras.layers.convolutional.Conv2D object at 0x00000115C1977630> True
<tensorflow.python.keras._impl.keras.layers.convolutional.Conv2D object at 0x000001159C56E8D0> True
<tensorflow.python.keras._impl.keras.layers.pooling.MaxPooling2D object at 0x00000115C1977B38> True
<tensorflow.python.keras._impl.keras.layers.convolutional.Conv2D object at 0x00000115C1977A58> True
<tensorflow.python.keras._impl.keras.layers.convolutional.Conv2D object at 0x00000115C19A9400> True
<tensorflow.python.keras._impl.keras.layers.pooling.MaxPooling2D object at 0x00000115C5E7F668> True
<tensorflow.python.keras._impl.keras.layers.convolutional.Conv2D object at 0x00000115C5E7FFD0> True
<tensorflow.python.keras._impl.keras.layers.convolutional.Conv2D object at 0x00000115D7A4E940> True
<tensorflow.python.keras._impl.ke

In [13]:
x = predictions/predictions.max()
X = predictions_test/predictions_test.max()

In [14]:
score = auc_svm(x,y,X,y_test, plot = False)

2


In [15]:
score

0.6321097602735108